Reading Data

In [69]:

import time
import re
import os
import pandas as pd 
import numpy as np
import fastai
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

In [70]:
train_df = pd.read_csv("/Users/rakeshravi/Downloads/covid19-global-forecasting-week-1/train.csv")
test_df = pd.read_csv("/Users/rakeshravi/Downloads/covid19-global-forecasting-week-1/test.csv")

Functions


In [71]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)
        
# def create_submission(test_df,model):

In [72]:
train_df.drop(["Id"], axis = 1, inplace = True)

In [73]:
for col in ["Country/Region","Province/State","Date"]:
    train_df[col] = train_df[col].astype("str")
    test_df[col] = test_df[col].astype("str")
train_df = train_df.sort_values("Date")

In [74]:
l_country = LabelEncoder()
train_df['Country/Region'] = l_country.fit_transform(train_df['Country/Region'])
l_state = LabelEncoder()
train_df['Province/State'] = l_state.fit_transform(train_df['Province/State'])
l_date = LabelEncoder()
train_df['Date'] = l_date.fit_transform(train_df['Date'])

In [75]:
train_df["Lat"].fillna(0, inplace = True)
train_df["Long"].fillna(0, inplace = True)
train_set_df =  train_df[:10000]
val_set_df = train_df[10000:]

In [76]:
X_train,y1_train,y2_train = train_set_df.drop(["ConfirmedCases","Fatalities"], axis = 1),train_set_df["ConfirmedCases"],train_set_df["Fatalities"]
X_val,y1_val,y2_val = val_set_df.drop(["ConfirmedCases","Fatalities"], axis = 1),val_set_df["ConfirmedCases"],val_set_df["Fatalities"]


In [77]:
# Set the random state for reproducibility
model_rf_1 = RandomForestRegressor(n_estimators=400, random_state=42)
model_rf_1.fit(X_train,y1_train)
model_rf_2 = RandomForestRegressor(n_estimators=400, random_state=42)
model_rf_2.fit(pd.concat([X_train,y1_train],axis = 1),y2_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [78]:
y1_preds = round(pd.DataFrame(model_rf_1.predict(X_val), columns = ["ConfirmedCases"]))
mean_squared_log_error(y1_val,y1_preds)

4.229308707934567

In [79]:
y2_preds = pd.DataFrame(model_rf_2.predict(pd.concat([X_val,y1_val], axis = 1)), columns = ["Fatalities"])
mean_squared_log_error(y2_val, y2_preds)

0.42914999357028527

In [82]:
import pandas as pd
from fbprophet import Prophet


ModuleNotFoundError: No module named 'fbprophet'

In [81]:
!pip install fbprophet

    100% |████████████████████████████████| 61kB 1.1MB/s ta 0:00:01
    100% |████████████████████████████████| 37.4MB 1.4MB/s eta 0:00:01  7% |██▌                             | 3.0MB 1.3MB/s eta 0:00:26
    100% |████████████████████████████████| 112kB 13.3MB/s ta 0:00:01
    100% |████████████████████████████████| 235kB 8.9MB/s eta 0:00:01
    100% |████████████████████████████████| 1.5MB 6.9MB/s ta 0:00:011    33% |██████████▉                     | 512kB 6.4MB/s eta 0:00:01
    100% |████████████████████████████████| 737kB 9.8MB/s eta 0:00:01
  Running setup.py bdist_wheel for fbprophet ... error
  Complete output from command /anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/fx/sw2mmdc15f1d08j04vg200fw0000gn/T/pip-install-f_jck06a/fbprophet/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/fx/sw2mmdc15f1d08j04vg200fw0000g

  Running setup.py install for fbprophet ... error
    Complete output from command /anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/fx/sw2mmdc15f1d08j04vg200fw0000gn/T/pip-install-f_jck06a/fbprophet/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/fx/sw2mmdc15f1d08j04vg200fw0000gn/T/pip-record-ebmph2oa/install-record.txt --single-version-externally-managed --compile:
    running install
    running build
    running build_py
    creating build
    creating build/lib
    creating build/lib/fbprophet
    creating build/lib/fbprophet/stan_model
    Importing plotly failed. Interactive plots will not work.
    INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_888a84912910fa0a45b9d614b75bb8a8 NOW.
    error: command 'gcc' failed with exit status 1
    
    ----------------------------------------
Command "/anaconda3/bin